# Jobagent Investigation
1. Run all cells.
1. View report at the bottom.

In [ ]:
su = "su3"
start = "2019-08-07T10:55:00.0000000Z"
end = "2019-08-07T11:30:36.0000000Z"
url = "https://notebooksv2.azure.com/yaananth/projects/06OasuNRs6rK/delays.ipynb"
baseUrl = "https://notebooksv2.azure.com/yaananth/projects/06OasuNRs6rK"
service = "tfs"
hub = "Build"
locationName = "tfsprodeus2su3"

In [ ]:
%%capture
!pip install nimport azure-kusto-notebooks

In [ ]:
# Import the things we use

# Note you can also use kql https://docs.microsoft.com/en-us/azure/data-explorer/kqlmagic
# %kql is single line magic
# %%kql is cell magic

# https://nbviewer.jupyter.org/github/ipython/ipython/blob/4.0.x/examples/IPython%20Kernel/Rich%20Output.ipynb#HTML
# https://ipython.readthedocs.io/en/stable/inte/magics.html
from IPython.display import display, HTML, Markdown, Javascript, clear_output

# http://pandas-docs.github.io/pandas-docs-travis/user_guide/reshaping.html
import pandas as pd
pd.options.display.html.table_schema = True
from pandas import Series, DataFrame
from datetime import datetime, timedelta, timezone
from urllib.parse import urlencode, quote_plus
from requests.utils import requote_uri
import time
import numpy as np
from matplotlib import pyplot as plt
from nimport.utils import tokenize, open_nb
import json
import os
import calendar as cal
import concurrent.futures
from azure.kusto.notebooks import utils as akn

In [ ]:
params = {
    "su": su,
    "start": start,
    "end": end,
    "url": url,
    "baseUrl": baseUrl,
    "service": service
}
root = 'devops-pipelines' if os.path.basename(os.getcwd()) != 'devops-pipelines' else ''
queryPath = os.path.join(root, 'queries')   

In [ ]:
# authenticate kusto client
# you will need to copy the token into a browser window for AAD auth. 
client = akn.get_client('https://vso.kusto.windows.net', 'VSO')

In [ ]:
q_slow = os.path.join(queryPath, "SlowActivities.csl")
q_sqlSlow = os.path.join(queryPath, "SlowSql.csl")

jaPath = os.path.join(queryPath, 'ja')
q_jobSql = os.path.join(jaPath, "JASqlTime.csl")
with concurrent.futures.ThreadPoolExecutor() as executor:
    p1 = executor.submit(akn.execute_file, client, 'VSO', q_slow, params)
    p2 = executor.submit(akn.execute_file, client, 'VSO', q_sqlSlow, params)
    p3 = executor.submit(akn.execute_file, client, 'VSO', q_jobSql, params)

q_slowResult_df = akn.to_dataframe_from_future(p1)

q_sqlSlowResult_df = akn.to_dataframe_from_future(p2)

q_jobSqlResult_df = akn.to_dataframe_from_future(p3)    


In [ ]:
print('=' * 50)
print('Report!')
print('=' * 50, '\n\n')

jarvisParams = {'su': su, 'start': akn.get_time(start, -10), 'end': akn.get_time(end, 10), 'service': service }

jaJarvisLink = """https://jarvis-west.dc.ad.msft.net/dashboard/VSO-ServiceInsights/PlatformViews/Compute-JA""" \
    """?overrides=[{"query":"//*[id='Service']","key":"value","replacement":"%(service)s"},""" \
    """{"query":"//*[id='RoleInstance']","key":"value","replacement":""},""" \
    """{"query":"//*[id='ScaleUnit']","key":"value","replacement":"%(su)s"}]""" \
    """&globalStartTime=%(start)s&globalEndTime=%(end)s&pinGlobalTimeRange=true""" % jarvisParams;
print('Jarvis dashboard link for job agents:\n', requote_uri(jaJarvisLink), '\n')

print('Top slow activities:')
display(q_slowResult_df)

print('Top sql slow activities:')
display(q_sqlSlowResult_df)

print('Top sql executime times from jobs:')
display(q_jobSqlResult_df)